In [4]:
%pip install azure-storage-blob azure-identity azure-ai-documentintelligence azure.ai.formrecognizer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.storage.blob import BlobServiceClient, BlobClient
import os

output_directory = "./txt/"

def doing(file_name):

    # Azure Document Intelligence  서비스의 엔드포인트와 키, 모델 이름
    form_recognizer_endpoint = "https://globalaibootcamp-s1-01.cognitiveservices.azure.com/" # #메모했던 내용으로 수정
    form_recognizer_key = "75b7320edf304e52981f453cfda924ad" # #메모했던 내용으로 수정
    model_id = "first_model" #메모했던 내용으로 수정

    # Azure Blob Storage 계정 정보
    blob_account_name = "gabs101" #메모했던 내용으로 수정
    blob_account_key = "BP3TFJ+rMuxxCq/zDutEjqVCuuAfjijzEaUEe07wwsofI2XTpZ/+w0hRxMEV2wQYYIlCDvV7Ubez+AStoDXWeA==" # #메모했던 내용으로 수정

    # Blob Storage 클라이언트 생성
    blob_service_client = BlobServiceClient(
        account_url=f"https://{blob_account_name}.blob.core.windows.net",
        credential=blob_account_key
    )

    # Blob의 일반 링크
    # #blob에서 예제 1의 url : https://계정이름.blob.core.windows.net/gabs101/예제1.pdf 이렇게 되어 있는데 https://계정이름.blob.core.windows.net/gabs101/ 이걸 붙여넣기
    blob_url = f'https://gabs101.blob.core.windows.net/gabs101/{file_name}' # #메모했던 예제1  URL 일부 내용으로 수정. /{file_name}은 그대로 둬 주세요

    # Blob 다운로드
    blob_client = BlobClient.from_blob_url(blob_url, credential=blob_account_key)
    blob_data = blob_client.download_blob().readall()

    # Document Analysis 클라이언트 생성
    document_analysis_client = DocumentAnalysisClient(
        endpoint=form_recognizer_endpoint, credential=AzureKeyCredential(form_recognizer_key)
    )



    # Make sure your document's type is included in the list of document types the custom model can analyze
    poller = document_analysis_client.begin_analyze_document(model_id, blob_data)
    result = poller.result()

    txt_file_name = os.path.join(output_directory, f"{os.path.splitext(file_name)[0]}.txt")

    for document in result.documents:
        with open(txt_file_name, "w") as file:
            for name, field in document.fields.items():
                field_value = field.value if field.value else field.content
                print("{}:'{}'".format(name, field_value))
                file.write(f"{name}: '{field_value}'\n")

   

In [13]:
doing("예제1.pdf")

reason:'걍 참 가 했 음'
learnt:'Thank u so much 여 러 분'
information:'{'name': DocumentField(value_type=dictionary, value={'ROW1': DocumentField(value_type=string, value='개 구 리', content=개구리, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=1.7, y=5.905), Point(x=2.31, y=5.905), Point(x=2.31, y=6.155), Point(x=1.7, y=6.155)])], spans=[DocumentSpan(offset=136, length=3)], confidence=None)}, content=None, bounding_regions=[], spans=[], confidence=None), 'birth': DocumentField(value_type=dictionary, value={'ROW1': DocumentField(value_type=string, value='2023/03/23', content=2023/03/23, bounding_regions=[BoundingRegion(page_number=1, polygon=[Point(x=3.51, y=5.92), Point(x=4.655, y=5.92), Point(x=4.655, y=6.17), Point(x=3.51, y=6.17)])], spans=[DocumentSpan(offset=140, length=10)], confidence=None)}, content=None, bounding_regions=[], spans=[], confidence=None), 'phone': DocumentField(value_type=dictionary, value={'ROW1': DocumentField(value_type=string, value='010-2555-8999', con

In [39]:
import re

def parse_file(file_path):
    with open(file_path, "r") as file:
        memo = file.read()


        reason_section = re.search(r"reason: '(.*?)'", memo)
        learnt_section = re.search(r"learnt: '(.*?)'", memo)
        information_section_match = re.search(r"information: '{(.+?)}'", memo) 
        satis_survey_section = re.search(r"satis_survey:\s*'\[(.*?)\]'", memo) 

        information_data = {}


        if information_section_match:
                            information_text = information_section_match.group(1)  # Extract key
                            print("txt : "+str(information_section_match))

                            # Improved regex for nested dictionaries
                            info_pattern = r"'(\w+)': DocumentField\(value_type=dictionary, value={'ROW1': DocumentField\(value_type=string, value='(.*?)', content=(.*?),"

                            

                            # Use information_text directly in finditer
                            for match in re.finditer(info_pattern, information_text):
                                key = match.group(1)
            
                                content = match.group(3).strip(",")
                                information_data[key] = content
                


        satis_survey_data = []
        if satis_survey_section:
            satis_survey_text = satis_survey_section.group(1)
            for match in re.finditer(r"\{.*?\}", satis_survey_text):
                value = match.group()
                satis_match = re.search(r"'satis':\s*DocumentField.*?value='(.*?)'", value)
                dissatis_match = re.search(r"'dissatis':\s*DocumentField.*?value='(.*?)'", value)
                satis = satis_match.group(1).strip() if satis_match else None
                dissatis = dissatis_match.group(1).strip() if dissatis_match else None
                satis_survey_data.append({'satis': satis, 'dissatis': dissatis})

    return {
        'reason': reason_section.group(1) if reason_section else None,
        'learnt': learnt_section.group(1) if learnt_section else None,
        'information': information_data,
        'satis_survey': satis_survey_data
    }


In [24]:
test_files=["예제1", "예제2"]
'''
for i in test_files:
    doing(str(i)+".pdf")'''

'\nfor i in test_files:\n    doing(str(i)+".pdf")'

In [41]:
import json

for i in test_files:
    txt_file_path = "./txt/"+str(i)+".txt"

    # 함수 호출 및 결과 출력
    parsed_data = parse_file(txt_file_path)
    print(parsed_data)
    
    # 딕셔너리를 JSON 형식의 문자열로 변환
    json_data = json.dumps(parsed_data, ensure_ascii=False)

    # 파일을 쓰기 모드로 열기 (파일이 없으면 생성됨)
    with open(txt_file_path, "w", encoding="utf-8") as file:
        # JSON 형식의 문자열을 파일에 쓰기
        file.write(json_data)

0
<re.Match object; span=(1093, 2349), match="information: '{'name': DocumentField(value_type=d>
txt : <re.Match object; span=(1093, 2349), match="information: '{'name': DocumentField(value_type=d>
{'reason': '걍 참 가 했 음', 'learnt': 'Thank u so much 여 러 분', 'information': {'name': '개구리', 'birth': '2023/03/23', 'phone': '010-2555-8999'}, 'satis_survey': [{'satis': 'V', 'dissatis': None}, {'satis': 'V', 'dissatis': None}]}
0
<re.Match object; span=(80, 1338), match="information: '{'name': DocumentField(value_type=d>
txt : <re.Match object; span=(80, 1338), match="information: '{'name': DocumentField(value_type=d>
{'reason': '인 공 지 능 에 대 해 서 배 우 고 싶 어 서 요', 'learnt': '인 공 지 능 을 앞 으 로 만 들 수 있 는 능 력', 'information': {'name': '옥토캣', 'birth': '2008/02/08', 'phone': '010-8888-8888'}, 'satis_survey': [{'satis': '√', 'dissatis': None}, {'satis': '√', 'dissatis': None}]}


In [ ]:
import os
import csv
import json

# 경로 설정
directory = './txt/'
output_file = 'output.csv'

# CSV 파일에 저장할 헤더
csv_headers = ['reason', 'learnt', 'name', 'birth', 'phone', 'satis_survey1', 'satis_survey2']

# CSV 파일 열기 및 헤더 쓰기
with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_headers)
    writer.writeheader()

    # 디렉토리 내의 모든 파일에 대해 반복
    for filename in os.listdir(directory):
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            # 파일 열기 및 JSON 데이터 로드
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                row = {}
                # 필요한 데이터 추출
                row['reason'] = data.get('reason', '')
                row['learnt'] = data.get('learnt', '')
                row['name'] = data['information']['name']
                row['birth'] = data['information']['birth']
                row['phone'] = data['information']['phone']
                # satis_survey가 null이 아닌 경우에만 값을 할당
                if data['satis_survey'][0]['satis'] is not None:
                    row['satis_survey1'] = data['satis_survey'][0]['satis']
                else:
                    row['satis_survey1'] = ''
                if data['satis_survey'][1]['satis'] is not None:
                    row['satis_survey2'] = data['satis_survey'][1]['satis']
                else:
                    row['satis_survey2'] = ''
                # CSV 파일에 데이터 쓰기
                writer.writerow(row)


In [43]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
    return chardet.detect(rawdata)['encoding']

file_path = '/workspaces/azureappdeploy/txt/예제2.txt'  # 파일 경로를 적절하게 변경하세요
encoding = detect_encoding(file_path)
print("파일의 인코딩:", encoding)

파일의 인코딩: utf-8
